# Redis backup and migration

This notebook walks through saving the state of a Redis database and restoring it on another instance by providing the target IP address and port. The workflow serialises every key using the native Redis commands for its data type (strings, hashes, lists, sets, sorted sets) while preserving TTL metadata and falling back to `DUMP`/`RESTORE` for unsupported types.


## How to use this notebook

1. Update the connection configurations (`SOURCE_REDIS_CONFIG` and `TARGET_REDIS_CONFIG`) with the IP address, port, database, and credentials for the instances you want to use.
2. Run the **Connection check** section to confirm access to the databases.
3. Execute the **Create backup** section to generate a JSON file containing the entire state of the source Redis instance.
4. Optionally transfer the file to another machine.
5. Run the **Restore backup** section to recreate the data on the target Redis instance.

⚠️ Cells that interact with Redis perform real operations on the database: enable the `FLUSH_TARGET_BEFORE_RESTORE` option carefully.


In [ ]:
from pathlib import Path
from typing import Any, Dict

from redis.exceptions import ConnectionError, ResponseError

from redis_backup import (
    DEFAULT_REDIS_DATABASES,
    create_multi_database_backup,
    display_backup_summary,
    get_database_overview,
    load_multi_database_backup_from_directory,
    restore_multi_database_backup,
    save_multi_database_backup_to_directory,
)


In [ ]:
# Redis instance configuration
SOURCE_REDIS_CONFIG: Dict[str, Any] = {
    "host": "127.0.0.1",  # Source instance IP or hostname
    "port": 6379,
    "db": 0,
    "username": None,  # Set a value if the server requires ACL authentication
    "password": None,
}

TARGET_REDIS_CONFIG: Dict[str, Any] = {
    "host": "127.0.0.1",  # Target instance IP or hostname
    "port": 6380,
    "db": 0,
    "username": None,
    "password": None,
}

DATABASES_TO_MIGRATE = tuple(DEFAULT_REDIS_DATABASES)  # Databases 0 through 10
BACKUP_DIRECTORY = Path("redis_backups")  # Directory where backups will be stored
SCAN_COUNT = 1000  # Number of keys inspected per iteration during the SCAN
FLUSH_TARGET_BEFORE_RESTORE = False  # Set True to flush the target DB before restoring


In [ ]:
# Helper functions are provided by the `redis_backup` module.
# You can open the `redis_backup.py` file to review the implementation or
# reuse them in other Python scripts.


In [ ]:
# Connection check (optional but recommended)
for label, configuration in (("Source", SOURCE_REDIS_CONFIG), ("Target", TARGET_REDIS_CONFIG)):
    try:
        overview = get_database_overview(configuration)
        print(
            f"{label}: Redis {overview['redis_version']} (mode {overview['redis_mode']}) - "
            f"{overview['key_count']} keys in DB {overview['db']}"
        )
    except ConnectionError as error:
        print(f"{label}: unable to connect -> {error}")


In [ ]:
# Create the backup from the source Redis instance
try:
    backup_payloads = create_multi_database_backup(
        SOURCE_REDIS_CONFIG,
        databases=DATABASES_TO_MIGRATE,
        scan_count=SCAN_COUNT,
    )
    save_multi_database_backup_to_directory(
        backup_payloads,
        BACKUP_DIRECTORY,
        file_prefix="redis_backup",
    )
    total_keys = sum(payload["metadata"]["key_count"] for payload in backup_payloads.values())
    print(
        f"Backup completed: {total_keys} keys saved across {len(backup_payloads)} databases."
    )
    print(f"Files generated in: {BACKUP_DIRECTORY.resolve()}")
    for db, payload in sorted(backup_payloads.items()):
        print()
        print(f"Database {db}:")
        display_backup_summary(payload)
except ConnectionError as error:
    print(f"Source Redis connection failed: {error}")
except ResponseError as error:
    print(f"Error during export: {error}")


In [ ]:
# Restore the backup on the target Redis instance
try:
    backup_payloads = load_multi_database_backup_from_directory(BACKUP_DIRECTORY)
    restored_counts = restore_multi_database_backup(
        backup_payloads,
        TARGET_REDIS_CONFIG,
        flush_each=FLUSH_TARGET_BEFORE_RESTORE,
    )
    total_restored = sum(restored_counts.values())
    print(
        f"Restore completed: {total_restored} keys imported across {len(restored_counts)} databases."
    )
    for db, restored in sorted(restored_counts.items()):
        print(f"  Database {db}: {restored} keys restored")
except FileNotFoundError:
    print(f"Backup directory not found: {BACKUP_DIRECTORY}")
except ConnectionError as error:
    print(f"Target Redis connection failed: {error}")
except ResponseError as error:
    print(f"Error during restore: {error}")
